# Section4：応用モデル

# 提案⼿法
- ディープラーニングモデルは精度は良いが、その分ネットワークが深くなり計算量が増える。
- 計算量が増えると、多くの計算リソースが必要で、お⾦がかかってしまう。
- ディープラーニングモデルの軽量化・⾼速化・⾼精度化を実現 (その名の通りモバイルなネットワーク)

## 一般的な畳み込みレイヤー

入力マップ：$H \times W \times C$ \
カーネル：$K \times K \times C$ \
出力マップ：$H \times W \times M$ 

ストライド１でパディングありの場合の１つの点での計算量は、$K \times K \times C \times M$ \
出力マップ全体すると、$H \times W \times K \times K \times C \times M$ 

## 4.1 MobileNets
Depthwise ConvolutionとPointwise Convolutionの組み合わせで軽量化を実現する

### Depthwise Convolution
-  仕組み
 - ⼊⼒マップのチャネルごとに畳み込みを実施
 - 出⼒マップをそれらと結合 (⼊⼒マップのチャネル数と同じになる)
 
チャンネルごとに畳み込みをするため、チャンネル間の関連性は考慮されない。

入力マップ：$H \times W \times C$ \
カーネル：$K \times K \times 1$ （フィルタ数は１固定）\
出力マップ：$H \times W \times C$

出力マップの計算量は：$H \times W \times C \times K \times K$

### Pointwise Convolution
- 仕組み
 -  1 x 1 convとも呼ばれる (正確には1 x 1 x c)
 - ⼊⼒マップのポイントごとに畳み込みを実施
 - 出⼒マップ(チャネル数)はフィルタ数分だけ作成可能 (任意のサイズが指定可能)
 
入力マップ：$H \times W\times C$ \
カーネル：$1 \times 1 \times C$ \
出力マップ：$H \times W \times M$

出力マップの計算量は：$H\times W\times C\times M$

### MobileNetのアーキテクチャ
- Depthwise Separable Convolutionという⼿法を⽤いて計算量を削減している。通常の畳込みが空間⽅向とチャネル⽅向の計算を同時に⾏うのに対して、Depthwise Separable ConvolutionではそれらをDepthwise ConvolutionとPointwise Convolutionと呼ばれる演算によって個別に⾏う。
- Depthwise Convolitionはチャネル毎に空間⽅向へ畳み込む。すなわち、チャネル毎に$D_K \times D_K \times 1$のサイズのフィルターをそれぞれ⽤いて計算を⾏うため、その計算量は$H \times W \times C \times D_K \times D_K $となる。
- 次にDepthwise Convolutionの出⼒をPointwise Convolutionによってチャネル⽅向に畳み込む。すなわち、出⼒チャネル毎に$1 \times 1 \times M$サイズのフィルターをそれぞれ⽤いて計算を⾏うため、その計算量は$H \times W \times C \times M$となる。

## 4.2 DenseNet
Dense Convolutional Network（以下、DenseNet）は、畳込みニューラルネットワーク（以下、CNN）アーキテクチャの⼀種である。ニューラルネットワークでは層が深くなるにつれて、学習が難しくなるという問題があったが、Residual Network（以下、ResNet）などのCNNアーキテクチャでは前⽅の層から後⽅の層へアイデンティティ接続を介してパスを作ることで問題を対処した。DenseBlockと呼ばれるモジュールを⽤いた、DenseNetもそのようなアーキテクチャの⼀つである。

- 初期の畳み込み
- Denseブロック
- 変換レイヤー
- 判別レイヤー

### Dense Block

#### 出⼒層に前の層の⼊⼒を⾜しあわせる
- 層間の情報の伝達を最⼤にするために全ての同特徴量サイズの層を結合する

#### 特徴マップの⼊⼒に対し、下記の処理で出⼒を計算
- Batch正規化
- Relu関数による変換
- 3 x 3畳み込み層による処理

#### kをネットワークのgrowth rateと呼ぶ
- kが⼤きくなるほど、ネットワークが⼤きくなるため、⼩さな整数に設定するのがよい

###  Transition Layer
- CNNでは中間層でチャネルサイズを変更し
- 特徴マップのサイズを変更し、ダウンサンプリングを⾏うため、Transition Layerと呼ばれる層でDence blockをつなぐ

### DenseNetとResNetの違い
- DenseBlockでは前⽅の各層からの出⼒全てが後⽅の層への⼊⼒として⽤いられる
- RessidualBlockでは前1層の⼊⼒のみ後⽅の層へ⼊⼒

### 正規化

#### Batch Norm
- ミニバッチに含まれるsampleの同⼀チャネルが同⼀分布に従うよう正規化
 - H x W x CのsampleがN個あった場合に、N個の同⼀チャネルが正規化の単位
 - RGBの3チャネルのsampleがN個の場合は、それぞれのチャンネルの平均と分散を求め正規化を実施 (図の⻘い部分に対応)。チャンネルごとに正規化された特徴マップを出⼒。
 - Batch Normalizationはニューラルネットワークにおいて学習時間の短縮や初期値への依存低減、過学習の抑制など効果がある。
- 問題点
 - Batch Sizeが⼩さい条件下では、学習が収束しないことがあり、代わりにLayer Normalizationなどの正規化⼿法が使われることが多い。
 
#### Layer Norm
- それぞれのsampleの全てのpixelsが同⼀分布に従うよう正規化
 -  N個のsampleのうち⼀つに注⽬。H x W x Cの全てのpixelが正規化の単位。
 -  RGBの3チャネルのsampleがN個の場合は、あるsampleを取り出し、全てのチャネルの平均と分散を求め正規化を実施 (図の⻘い部分に対応)。特徴マップごとに正規化された特徴マップを出⼒
 -  ミニバッチの数に依存しないので、上記の問題を解消できていると考えられる。
 
#### Instance Nrom
- さらにchannelも同⼀分布に従うよう正規化
 - 各sampleをチャンネルを ごとに正規化
 - コントラストの正規化に寄与・画像のスタイル転送やテクスチャ合成タスクなどで利用


## 4.3 Wavenet
- 生の音声波形を生成する深層学習モデル
- Pixel CNN（高解像度の画像を精密に生成できる手法）を音声に応用したもの
- 時系列データに対して畳み込み（Dilated convolution）を適用する。
- Dilated convolution
 - 層が深くなるにつれて畳み込むリンクを離す


# 確認テスト

#### P86
問）
- Depthwise Convolitionはチャネル毎に空間⽅向へ畳み込む。すなわち、チャネル毎に$D_K \times D_K \times 1$のサイズのフィルターをそれぞれ⽤いて計算を⾏うため、その計算量は（い）となる。
- 次にDepthwise Convolutionの出⼒をPointwise Convolutionによってチャネル⽅向に畳み込む。すなわち、出⼒チャネル毎に$1 \times 1 \times M$サイズのフィルターをそれぞれ⽤いて計算を⾏うため、その計算量は（う）となる。

答） 

- （い）：$H \times W \times C \times D_K \times D_K $
- （う）：$H \times W \times C \times M$

#### P106
深層学習を⽤いて結合確率を学習する際に、効率的に学習が⾏えるアーキテクチャを提案したことが WaveNet の⼤きな貢献の1 つである。提案された新しい Convolution 型アーキテクチャは（あ）と呼ばれ、結合確率を効率的に学習できるようになっている。

- Dilated causal convolution
- Depthwise separable convolution　⇒MobileNetで使われている
- Pointwise convolution　　　　　　 ⇒MobileNetで使われている
- Deconvolution　　　　　　　　　　⇒逆畳み込み演算

（あ）を⽤いた際の⼤きな利点は、単純な Convolution layer と⽐べて（い）ことである。

- パラメータ数に対する受容野が広い
- 受容野あたりのパラメータ数が多い　　⇒パラメータ数は減っているため違う
- 学習時に並列計算が⾏える　　　　　⇒並列演算はできない。
- 推論時に並列計算が⾏える　　　　　⇒並列演算はできない。

答） 

- （あ）：Dilated causal convolution
- （い）：パラメータ数に対する受容野が広い